In [6]:
! pip install --upgrade keras
! pip install --upgrade tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.8/208.8 MB 1.5 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow-macos
    Found existing installation: tensorflow-macos 2.14.0
    Uninstalling tensorflow-macos-2.14.0:
      Successfully uninstalled tensorflow-macos-2.14.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.14.0
    Uninstalling tensorflow-2.14.0:
      Successfully uninstalled t

In [9]:
pip show tensorflow


Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/saisindhusangavi/anaconda3/lib/python3.10/site-packages
Requires: tensorflow-macos
Required-by: tf-models-official
Note: you may need to restart the kernel to use updated packages.


In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import pickle
import numpy as np
import os

file = open("/Users/saisindhusangavi/Desktop/ML LAB/christmascarol.txt", "r", encoding = "utf8")
 
# store file in list
lines = []
for i in file:
    lines.append(i)
 
# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 
 
#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space
 
#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]
len(data)

tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
 
# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))
 
sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]
len(sequence_data)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)
sequences = []
 
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
     
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]
X = []
y = []
 
for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
     
X = np.array(X)
y = np.array(y)
print("Data: ", X[:10])
print("Response: ", y[:10])
y = to_categorical(y, num_classes=vocab_size)
y[:5]

model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

from tensorflow import keras

plot_model(model, to_file='plot.png', show_layer_names=True)

from tensorflow.keras.callbacks import ModelCheckpoint
 
checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=30, batch_size=64, callbacks=[checkpoint])



4843
The Length of sequences are:  32771
Data:  [[  1  51 109]
 [ 51 109 306]
 [109 306   3]
 [306   3   4]
 [  3   4  47]
 [  4  47 331]
 [ 47 331  27]
 [331  27 306]
 [ 27 306  32]
 [306  32  17]]
Response:  [306   3   4  47 331  27 306  32  17   1]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             48430     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 4843)              4847843   
  

Epoch 1/30
513/513 [==============================] - ETA: 0s - loss: 6.7179
Epoch 1: loss improved from inf to 6.71792, saving model to next_words.h5
513/513 [==============================] - 54s 102ms/step - loss: 6.7179
Epoch 2/30


/Users/saisindhusangavi/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


513/513 [==============================] - ETA: 0s - loss: 6.2292
Epoch 2: loss improved from 6.71792 to 6.22917, saving model to next_words.h5
513/513 [==============================] - 52s 102ms/step - loss: 6.2292
Epoch 3/30
513/513 [==============================] - ETA: 0s - loss: 5.8224
Epoch 3: loss improved from 6.22917 to 5.82238, saving model to next_words.h5
513/513 [==============================] - 53s 103ms/step - loss: 5.8224
Epoch 4/30
513/513 [==============================] - ETA: 0s - loss: 5.4831
Epoch 4: loss improved from 5.82238 to 5.48314, saving model to next_words.h5
513/513 [==============================] - 53s 104ms/step - loss: 5.4831
Epoch 5/30
513/513 [==============================] - ETA: 0s - loss: 5.1971
Epoch 5: loss improved from 5.48314 to 5.19710, saving model to next_words.h5
513/513 [==============================] - 57s 112ms/step - loss: 5.1971
Epoch 6/30
513/513 [==============================] - ETA: 0s - loss: 4.9426
Epoch 6: loss improved

In [11]:
# Load the model and tokenizer
model = load_model('next_words.h5')
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             48430     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 4843)              4847843   
                                                                 
Total params: 17945273 (68.46 MB)
Trainable params: 17945273 (68.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
